In [1]:
import numpy as np
import random
from tqdm import tqdm
from copy import deepcopy

import torch 

from utils.reproduce_functions import *
from utils.mutation_functions import *
from utils.parent_selection_functions import *
from utils.other_fucntions import * 
from utils.network import *


import lab9_lib

In [2]:
PROBLEM_SIZE = 10

MU = 10
LAMBDA = 20
strategy = 'plus' # comma or plus

MUTATION_PROB = 0.2
DYNAMIC_MUTATION_PROB = True
DIVERSITY_THRESHOLD = 20

LENGTH_SOLUTION = 1000
NUMBER_GENERATIONS = 2_500

In [3]:
mutate = one_bit_flip
reproduce = uniform_crossover
parent_selection = roulette

## Network Tests

In [ ]:
n_dataset = 100_000
batch_size = 32
name = "a_lr_0.001_bs_32"

fitness = lab9_lib.make_problem(PROBLEM_SIZE)

dataset = generate_dataset(fitness, length_solution=LENGTH_SOLUTION, size=n_dataset)
train_dataloader, val_dataloader, test_dataloader = get_loaders(dataset, batch_size=batch_size)

In [ ]:
dataset[list(dataset.keys())[0]]

In [ ]:
device = torch.device('cuda')

net = FitnessNet_b().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

train(model=net, 
      criterion=criterion, 
      optimizer=optimizer, 
      train_dataloader=train_dataloader, 
      val_dataloader=val_dataloader, 
      test_dataloader=test_dataloader,
      device=device, 
      epochs=5)

save = True
if save:
    save_model(net, name)

In [ ]:
values, high, low = plot_results(model=net, test_dataloader=test_dataloader, device=device, name=name, save=save, criterion=criterion)

In [ ]:
# to test on a random element
net.eval()

value = torch.from_numpy(np.random.randint(2, size=LENGTH_SOLUTION)).float().unsqueeze(0).unsqueeze(1).to(device)
print(net(value), value[0][0][:10], fitness(value[0][0]))

In [ ]:
plot_high_low(high=high, low=low)

## GA

In [15]:
import torch 

def ga(fitness, parents, parents_evals, memoization=False, use_nn=False):
    generation_convergence = -1
    activate_nn = False
    nn_threshold = 10_000

    if use_nn:
        pop_nn_history = {}
        device = torch.device('cuda')
        net = FitnessNet_b().to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    
    if memoization:
        pop_history = {}
        for i in range(len(parents)):
            pop_history[parents[i].tobytes()] = (parents[i], parents_evals[i]) # CHANGED

    for generation in tqdm(range(NUMBER_GENERATIONS)):
        offsprings = []
        offsprings_evals = []
        while len(offsprings) < LAMBDA:
            # Parent Selection
            p1, p2 = parent_selection(parents, parents_evals) 

            # Reproduce Parents
            off_spring = reproduce(p1, p2)

            # Mutate Offspring
            if DYNAMIC_MUTATION_PROB:
                p_div = get_parents_diversity(p1, p2)
                new_ind1 = mutate(off_spring, \
                                  mutation_probability=(1 - (min(p_div,LENGTH_SOLUTION/2)/(LENGTH_SOLUTION/2))) )
            else:    
                new_ind1 = mutate(off_spring, mutation_probability=MUTATION_PROB)
            
            # CHANGED
            # Evaluate Offspring
            if memoization:
                new_ind1_eval = pop_history.get(new_ind1.tobytes())
                if new_ind1_eval == None:
                    if activate_nn:
                        new_ind1_eval = net(torch.from_numpy(new_ind1).float().unsqueeze(0).unsqueeze(1).to(device))
                        new_ind1_eval = new_ind1_eval.cpu().detach().numpy()[0][0]
                        pop_nn_history[new_ind1.tobytes()] = (new_ind1, new_ind1_eval)
                    else:
                        new_ind1_eval = fitness(new_ind1)
                        pop_history[new_ind1.tobytes()] = (new_ind1, new_ind1_eval)
                else: 
                    new_ind1_eval = new_ind1_eval[1]
            else:
                new_ind1_eval = fitness(new_ind1)
            # END CHANGED

            # Add it to new population
            offsprings.append(new_ind1)
            offsprings_evals.append(new_ind1_eval)

        # Create new population (with parents if plus, without if comma)
        all_people = (parents if strategy=='plus' else []) + offsprings
        all_evals = (parents_evals if strategy=='plus' else []) + offsprings_evals
        best_people = np.argsort(all_evals)[::-1]

        parents = []
        parents_evals = []
        for i in range(MU):
            parents.append(all_people[best_people[i]])
            parents_evals.append(all_evals[best_people[i]])
            
        if np.max(parents_evals) - 1.0 >= 0 and generation_convergence == -1:
            generation_convergence = generation, fitness.calls
        
        # CHANGED
        # at the end of each generation, if the number of known solutions is greater than the nn_threshold, activate the neural network
        if use_nn and len(pop_history) > nn_threshold and not activate_nn:
            activate_nn = True
            train_dataloader, val_dataloader, test_dataloader = get_loaders(pop_history, batch_size=32, train_size=1.0, val_size=0.0, test_size=0.0)
            print("Activating neural network")
            train(model=net, 
                  criterion=criterion, 
                  optimizer=optimizer, 
                  train_dataloader=train_dataloader, 
                  val_dataloader=val_dataloader, 
                  test_dataloader=test_dataloader,
                  device=device, 
                  epochs=20)
        

    # take the 10 best solutions in pop_nn_history and evaluate them with the fitness function 
    if use_nn:
        val = {}
        best_nn_solutions = np.argsort([pop_nn_history[key][1] for key in pop_nn_history])[::-1][:10]
        for i in best_nn_solutions:
            key = list(pop_nn_history.keys())[i]
            value = pop_nn_history[key]
            val[key] = (value[0], fitness(value[0]))
            print(f"Value: {val[key][1]}")
            # END CHANGED
        


    return parents, parents_evals, generation_convergence

In [10]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
parents = [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(50)]
parents_evals = [fitness(x) for x in parents]
parents, parents_evals, gc = ga(fitness, parents, parents_evals, memoization=True, use_nn=False)
i_best = np.argmax(parents_evals)
# print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)
print("Generation, NumCalls @ fitness=1.0 : ", gc)

100%|██████████| 2500/2500 [00:59<00:00, 42.07it/s]

Best score:  0.174
Num fitness calls:  17296
Generation, NumCalls @ fitness=1.0 :  -1


In [9]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
parents = [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(50)]
parents_evals = [fitness(x) for x in parents]
parents, parents_evals, gc = ga(fitness, parents, parents_evals, memoization=True, use_nn=True)
i_best = np.argmax(parents_evals)
# print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)
print("Generation, NumCalls @ fitness=1.0 : ", gc)

  0%|          | 0/2500 [00:00<?, ?it/s]

 20%|█▉        | 497/2500 [00:12<00:49, 40.41it/s]

Activating neural network


c:\ProgramData\miniconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\ProgramData\miniconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch:  0  Loss:  0.0015974001726135612


...

Epoch:  18  Loss:  0.0009313744376413524


 20%|██        | 504/2500 [00:28<27:27,  1.21it/s]

Epoch:  19  Loss:  0.0003991955891251564


100%|██████████| 2500/2500 [01:45<00:00, 23.69it/s]

Value: 0.10645555557
Value: 0.16745555569999998
Value: 0.16735556669999999
Value: 0.16744555560000002
Value: 0.050835555557
Value: 0.10645555557
Value: 0.16745555569999998
Value: 0.16745555569999998
Value: 0.1674455557
Value: 0.16744555560000002
Best score:  0.1675555556
Num fitness calls:  10027
Generation, NumCalls @ fitness=1.0 :  -1
